# Доверительный интервал

In [ ]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

#### z-интервал

Допустим, нам откуда-то известно, что дисперсия auc_scores $\sigma^2=0.25$. Построим доверительные интервалы для средних вида $$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

In [ ]:
print "95%% confidence interval", _zconfint_generic(mean,sqrt(0.25/n), alpha = 0.05, 'two-sided')

#### t-интервал

Вместо гипотетической теоретической дисперсии $\sigma^2$, которую мы на самом деле в данном случае не знаем, используем выборочные дисперсии, и построим доверительные интервалы вида $$\bar{X}_n \pm t_{1-\frac{\alpha}{2}} \frac{S}{\sqrt{n}}$$

In [ ]:
print "mean 95%% confidence interval", _tconfint_generic(mean, std/sqrt(n), dof= n - 1, alpha=0.05, 'two-sided')

# Доверительные интервалы для доли

In [ ]:
from statsmodels.stats.proportion import proportion_confint

### Доверительный интервал на основе нормального распределения

$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [ ]:
normal_interval = proportion_confint(sum(sample), len(sample), method = 'normal')
print 'normal_interval [%f, %f]' % (normal_interval[0], normal_interval[1]) 
                                                 

### Доверительный интервал Уилсона

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{
z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [ ]:
wilson_interval = proportion_confint(sum(sample), len(sample), method = 'wilson')
print 'wilson_interval [%f, %f]' % (wilson_interval[0], wilson_interval[1])

## Размер выборки для интервала заданной ширины

In [ ]:
from statsmodels.stats.proportion import samplesize_confint_proportion
import numpy as np
n_samples = int(np.ceil(samplesize_confint_proportion(sample.mean(), width/2)))
# n_samples - размер выборки
# width - ширина доверительного интервала

# Доверительные интервалы для двух долей

In [ ]:
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

## Доверительный интервал для разности долей (независимые выборки)

 $..$ | $X_1$ | $X_2$  
 ------------- | ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [ ]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(sample1, sample2)

## Доверительный интервал для разности долей (связанные выборки)

$X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------| -------------| -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [ ]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)
print "confidence interval: [%f, %f]" % proportions_confint_diff_rel(sample1, sample2)

# Доверительные интервалы на основе bootstrap

In [ ]:
def get_bootstrap_samples(data, n_samples): #функция генерации n_samples выборок из data, методом bootstrap
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha): #функция генерации доверительного интервала для статисики
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [ ]:
no_def_age = map(np.median, get_bootstrap_samples(df[df.default == 0].AGE.values, 1000)) #генерируем выборки и находим медианы
with_def_age = map(np.median, get_bootstrap_samples(df[df.default == 1].AGE.values, 1000))

print 'no_def_age median: ', sum(no_def_age) / float(len(no_def_age)) #считаем среденее значение медианы в 1000 выборках 
print 'with_def_age median: ', sum(with_def_age) / float(len(with_def_age))

delta_median_scores_age = map(lambda x: x[0] - x[1], zip(no_def_age, with_def_age)) # оценка разности медиан

print 'delta ', sum(delta_median_scores_age) / float(len(delta_median_scores_age)) #средняя разность

print "95% confidence interval for the difference between medians",  stat_intervals(delta_median_scores_age, 0.05) #доверительный интервал

# Проверка гипотез (ПАРАМЕТРИЧЕСКИЕ КРИТЕРИИ)

## Критерий Стьюдента

In [ ]:
import scipy
from statsmodels.stats.weightstats import *

## Одновыборочный критерий Стьюдента

𝐻0:  среднее значение выборки равно mean.

𝐻1: не равно.

In [ ]:
print stats.ttest_1samp(data, mean) #значение статистики и уровень значимости
print "95%% confidence interval: [%f, %f]" % zconfint(data) #доверительный интервал

## Двухвыборочный критерий Стьюдента (независимые выборки)

𝐻0:  средние значения двух независимых выборок одинаковые.

𝐻1:  не одинаковы.

In [ ]:
scipy.stats.ttest_ind(data_1, data_2, equal_var = False) #значение статистики и уровень значимости
cm = CompareMeans(DescrStatsW(data_1), DescrStatsW(data_2)) #сравнение средних
print "95%% confidence interval: [%f, %f]" % cm.tconfint_diff(usevar='unequal') #доверительный интервал для разности средних

# Двухвыборочный критерий Стьюдента (зависмые выборки)

𝐻0: средние значения двух независимых выборок одинаковые.

𝐻1: не одинаковы.

In [ ]:
stats.ttest_rel(data_1, data_2) #значение статистики и уровень значимости
print "95%% confidence interval: [%f, %f]" % DescrStatsW(data_1 - data_2).tconfint_mean() #доверительный интервал для разности средних

# Нормальность выборки

Проверка, что распределения в выборках существенно не отличаются от нормальных.

In [ ]:
# Q-Q график
%pylab inline
pylab.figure(figsize=(12,8))
pylab.subplot(2,2,1)
stats.probplot(data_1, dist="norm", plot=pylab)
pylab.subplot(2,2,2)
stats.probplot(data_2, dist="norm", plot=pylab)
pylab.show()

## Критерий Шапиро-Уилка:

𝐻0:  выборка распредлена нормально

𝐻1:  не нормально.

In [ ]:
print "Shapiro-Wilk normality test, W-statistic: %f, p-value: %f" % stats.shapiro(data_1)
print "Shapiro-Wilk normality test, W-statistic: %f, p-value: %f" % stats.shapiro(data_2)

# Проверка гипотез о долях

### Z-критерий для доли

In [ ]:
stats.binom_test(sum(data), n=len(data), p=0.05, alternative='greater') 
# n - число испытаний
# sum(data) - число положительных исходов

### Z-критерий для двух долей

In [ ]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

H0: p1=p2

H1: p1!=p2

In [ ]:
def proportions_diff_z_stat_ind(sample1, sample2): #вычисление значения z-статистики
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'): # вычисление уровня значимости
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

 ### Z-критерий для разности долей (связанные выборки)

 $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------| -------------| -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$


H0: p1=p2

H1: p1!=p2

In [ ]:
def proportions_diff_z_stat_rel(sample1, sample2): # вычисление z-статистики для разности 2х долей
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'): # вычисление уровня значимости
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

# Проверка гипотез (НЕПАРАМЕТРИЧЕСКИЕ КРИТЕРИИ)

 Критерий | Одновыборочный | Двухвыборочный | Двухвыборочный (связанные выборки)  
  ------------- | -------------| -------------|-------------|
  **Знаков**          |$\times$   |     $ $     | $\times$ 
  **Ранговый**        | $\times$  | $\times$ | $\times$  
  **Перестановочный** | $\times$  | $\times$ | $\times$ 

In [ ]:
from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

# КРИТЕРИИ ЗНАКОВ

### Критерий знаков (одновыборочный)
𝐻0: медиана  равна m0

𝐻1: медиана не равна m0

In [ ]:
print "M: %d, p-value: %f" % sign_test(data, m0)

### Критерий знаков (для связанных выборок)

$H_0\colon P\left(X_1>X_2\right)=\frac1{2},$

$H_1\colon P\left(X_1>X_2\right)\neq\frac1{2}$

In [ ]:
print "M: %d, p-value: %f" % sign_test(data_1 - data_2)

# РАНГОВЫЕ КРИТЕРИИ

### Критерий знаковых рангов Вилкоксона (одновыборочный)

𝐻0: медиана равна m0

𝐻1: медиана не равна m0

In [ ]:
stats.wilcoxon(data - m0, alternative='two-sided')

### Ранговый критерий Манна-Уитни (для 2х независимых выборок)

$H_0\colon F_{X_1}(x) = F_{X_2}(x)$

$H_1\colon F_{X_1}(x) = F_{X_2}(x + \Delta), \Delta\neq 0$

In [ ]:
stats.mannwhitneyu(data_1, data_2)

### Критерий знаковых рангов Вилкоксона (для связанных выборок)
$H_0\colon med\left(X_1-X_2\right)=0,$

$H_1\colon med\left(X_1-X_2\right)\neq0$

In [ ]:
stats.wilcoxon(data_1 - data_2)
#stats.wilcoxon(data_1, data_2) - то же самое

# ПЕРЕСТАНОВОЧНЫЕ КРИТЕРИИ

### Перестановочный критерий (одновыборочный)
𝐻0: среднее равно mean

𝐻1: среднее не равно mean

In [ ]:
def permutation_t_stat_1sample(sample, mean): #функция вычисления T-статистики
    t_stat = sum(map(lambda x: x - mean, sample))
    return t_stat

def permutation_zero_distr_1sample(sample, mean, max_permutations = None): #функция построения нулевого распределения
    centered_sample = map(lambda x: x - mean, sample)
    if max_permutations:
        signs_array = set([tuple(x) for x in 2 * np.random.randint(2, size = (max_permutations, 
                                                                              len(sample))) - 1 ])
    else:
        signs_array =  itertools.product([-1, 1], repeat = len(sample))
    distr = [sum(centered_sample * np.array(signs)) for signs in signs_array]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'): #функция вычисления уровня значимости
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_1sample(sample, mean)
    
    zero_distr = permutation_zero_distr_1sample(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print "p-value: %f" % permutation_test(data, mean)

### Перестановочный критерий (для 2х независимых выборок)
$H_0\colon F_{X_1}(x) = F_{X_2}(x)$

$H_1\colon F_{X_1}(x) = F_{X_2}(x + \Delta), \Delta\neq 0$

In [ ]:
def permutation_t_stat_ind(sample1, sample2): #функция вычисления T-статистики
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations): #функция генерирования индексов
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None): #функция построения нулевого распределения
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):  #функция вычисления уровня значимости
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print "p-value: %f" % permutation_test(data_1, data_2, max_permutations = 10000)

### Перестановочный критерий (для 2х связанных выборок)
$H_0\colon \mathbb{E}(X_1 - X_2) = 0$

$H_1\colon \mathbb{E}(X_1 - X_2) \neq 0$

In [ ]:
def permutation_t_stat_1sample(sample, mean):
    t_stat = sum(map(lambda x: x - mean, sample))
    return t_stat
def permutation_zero_distr_1sample(sample, mean, max_permutations = None):
    centered_sample = map(lambda x: x - mean, sample)
    if max_permutations:
        signs_array = set([tuple(x) for x in 2 * np.random.randint(2, size = (max_permutations, 
                                                                              len(sample))) - 1 ])
    else:
        signs_array =  itertools.product([-1, 1], repeat = len(sample))
    distr = [sum(centered_sample * np.array(signs)) for signs in signs_array]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_1sample(sample, mean)
    
    zero_distr = permutation_zero_distr_1sample(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print "p-value: %f" % permutation_test(data_1 - data_2, 0., max_permutations = 1000)

# Корреляции

### Корреляция Стьюдента и Спирмена

In [ ]:
dt.corr() #стьюдент
dt.corr(method='spearman') #спирмен

In [ ]:
from scipy.stats import pearsonr
corr, p = pearsonr(dt[col1], dt[col2]) #расчет корреляции пирсона с уровнем значимости

### Корреляция Мэтьюса

In [ ]:
mas = np.array([a,b],[c,d]) #таблица сопряженности бинарных признаков
def Metus_cor(a,b,c,d):
    return (a*d-b*c)/float(np.sqrt((a+b)*(a+c)*(d+b)*(c+d)))

### Проверка гипотезы значимости корреляции Мэтьюса (критерий хи-квадрат)

In [ ]:
from scipy.stats import chi2_contingency
obs = np.array([[a,b],[c,d]]) #таблица сопряженности бинарных признаков или категориальных
chi2, p, dof, ex = chi2_contingency(obs)
print 'stat=',chi2 #значение статистики
print 'p=', p #достигаемый уровень значимости
print 'dof=', dof #число степеней свободы
print 'ex=', ex #ожидаемые частоты обытий

### Критерий хи-квадрат

In [ ]:
from scipy.stats import chisquare
xi_2_f = chisquare(mas1.reshape(),mas2.reshape(), ddof=0) #гипотеза mas1 распределен по такому же закону, что и  mas2
xi_2_f #выдает статистику и уровень значмости

# Множественная проверка гипотез

## Поправки на множественную проверку
### Метод Холма

In [ ]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(df.p, alpha = 0.05, method = 'holm') 
# df.p - столбец df в котором записаны уровни значимости гипотез
# alpha - вероятности совершить хотя бы 1  ошибку 1 рода
# reject  - решение отвергнуть(true) гипотезу или нет(false)
# p_corrected - скректированные уровни значимости

### Метод Бенджамини-Хохберга

In [ ]:
reject, p_corrected, a1, a2 = multipletests(df.p, alpha = 0.05, method = 'fdr_bh') 